# Data Cleaning and Selection Patientendaten

## Imports and Read

In [52]:
import pandas as pd
import datetime
import numpy as np
import re

In [53]:
print("Start Clean and Preprocessing data patient")

Start Clean and Preprocessing data patient


In [54]:
df = pd.read_excel(r'../raw_data/Hypophysenpatienten.xlsx',sheet_name='no duplicate PID')

In [55]:
df.columns

Index(['%ID', 'Fall Nr.', 'Datum/Zeit', 'Modalität', 'Exam Code', 'Exam Name',
       'Abteilung', 'Arbeitsplatz.Kürzel', 'Aufnahmeart', 'PID', 'Grösse',
       'Ausfälle prä', 'Prolaktin', 'IGF1', 'Cortisol', 'fT4',
       'weiteres Labor', 'Qualität', 'ED', 'OP Datum', 'Ausfälle post',
       'Diagnose', 'Kategorie', 'Patient Alter', 'Zuweiser',
       'AnforderungDatum', 'ÜberweiserIntern.Bereich',
       'ÜberweiserIntern.Klinik'],
      dtype='object')

## Basic Cleaning, Column Selection, Anomaly Correction and Format definition

In [56]:
# define needed columns
column_list = ['PID',"Datum/Zeit","Arbeitsplatz.Kürzel",'Grösse',
       'Ausfälle prä', 'Qualität', 'ED','OP Datum', 'Ausfälle post',
       'Diagnose', 'Kategorie', 'Patient Alter',
       'Prolaktin',"IGF1", 'Cortisol','fT4','weiteres Labor']


### Data Type Definition


In [57]:
#TODO: check Tristan
# not parseable correct values corrected
df.loc[3,'ED'] = datetime.datetime(2006,1,1,0,0,0,0)
df.loc[12,'ED'] = datetime.datetime(2008,1,1,0,0,0,0)

#TODO: check Tristan
# correct a value which is not datetime parseable
df.loc[df['OP Datum'] == '2006, 2009', 'OP Datum'] = datetime.datetime(2006,1,1,0,0,0,0)

In [58]:
# make datetime values
df["Datum/Zeit"] = pd.to_datetime(df["Datum/Zeit"])
df["ED"] = pd.to_datetime(df["ED"])
df["OP Datum"] = pd.to_datetime(df["OP Datum"])

In [59]:
# TODO: anomaly? check tristan
# rows where Entry Date is after Operationdate?
df[df['OP Datum'] < df['ED']][['ED','OP Datum']]

,ED,OP Datum
20,2023-09-02,2009-02-18


### Impute NAN Lab Data

In [60]:
# TODO: set impute value
df.loc[df["Prolaktin"].isna(),'Prolaktin'] = '0'
df.loc[df["IGF1"].isna(),'IGF1'] = '0'
df.loc[df["Cortisol"].isna(),'Cortisol'] = '0'
df.loc[df["fT4"].isna(),'fT4'] = '0'


### Unit Conversion Lab Data

In [61]:
df["Prolaktin"].unique()

array(['0', '173mU/l', '743mU/l', '687mU/l', '7.8 ug/l'], dtype=object)

In [62]:
# get indices which need to be converted
indices_to_divide = df.loc[df["Prolaktin"].str.contains('ug/l'),'Prolaktin'].index 
# remove units and strings
df['Prolaktin'] = df['Prolaktin'].str.rstrip(r'mU/l')
df['Prolaktin'] = df['Prolaktin'].str.rstrip(r'ug/l')
df['Prolaktin'] = df['Prolaktin'].astype(float)
# ug/l -> mU/l (ug/l * 21.2)
df.loc[indices_to_divide,'Prolaktin'] = df.loc[indices_to_divide,'Prolaktin'] * 21.2


In [63]:
df["IGF1"].unique()

array(['0', '6.3nmol/l', '20.2nmol/l', '75.4ng/ml', '208 ng/ml'],
      dtype=object)

In [64]:
# get indices which need to be converted
indices_to_divide = df.loc[df["IGF1"].str.contains('ng/ml'),'IGF1'].index 
# remove units and strings
df['IGF1'] = df['IGF1'].str.rstrip(r'nmol/l')
df['IGF1'] = df['IGF1'].str.rstrip(r'ng/ml')
df['IGF1'] = df['IGF1'].astype(float)
# ng/ml -> nmol/l (ng/ml / 7.65)
df.loc[indices_to_divide,'IGF1'] = df.loc[indices_to_divide,'IGF1'] / 7.65


In [65]:
df["Cortisol"].unique()

array(['0', 329, 271, 110, '311 nmol/l'], dtype=object)

In [66]:
# # get indices which need to be converted
# indices_to_divide = df.loc[df["Cortisol"].str.contains('ng/ml'),'Cortisol'].index 
# # remove units and strings
# df['Cortisol'] = df['Cortisol'].str.rstrip(r'nmol/l')
# df['Cortisol'] = df['IGF1'].str.rstrip(r'ng/ml')
# df['Cortisol'] = df['Cortisol'].astype(float)
# # ng/ml -> nmol/l (ng/ml *  27.59)
# df.loc[indices_to_divide,'Cortisol'] = df.loc[indices_to_divide,'Cortisol'] / 7.65


In [67]:
df["fT4"].unique()

array(['0', 10.1, 8.4, 7.3, '14.6 pmol/l'], dtype=object)

In [68]:
# # get indices which need to be converted
# indices_to_divide = df.loc[df["fT4"].str.contains('ng/ml'),'fT4'].index 
# # remove units and strings
# df['fT4'] = df['fT4'].str.rstrip(r'nmol/l')
# df['fT4'] = df['fT4'].str.rstrip(r'ng/ml')
# df['fT4'] = df['fT4'].astype(float)
# # ng/ml -> nmol/l (ng/ml / 7.65)
# df.loc[indices_to_divide,'fT4'] = df.loc[indices_to_divide,'fT4'] / 7.65


In [69]:
# Patient ID Duplicate Check
assert len(df[df["PID"].duplicated()]) == 0

# Case Nr Duplicate Check
assert len(df[df["Fall Nr."].duplicated()]) == 0

In [70]:
# select and rename columns
df = df[column_list]
df= df.rename(columns={"Fall Nr.": "Case_ID","PID": "Patient_ID",
                       "Datum/Zeit": "Date_MRI","ED": "Entry_date", "OP Datum": "Operation_date",
                       "Arbeitsplatz.Kürzel":"ID_MRI_Machine","Grösse": "Adenoma_size","Qualität": "Label_Quality",
                       "Patient Alter":"Patient_age","Kategorie":"Category","Diagnose":"Diagnosis",
                       "Prolaktin":"Prolactin","weiteres Labor":"Lab_additional"})

In [71]:
# set category data type in pandas, check datatypes
df['ID_MRI_Machine'] = df['ID_MRI_Machine'].astype('category')
df['Adenoma_size'] = df['Adenoma_size'].astype('category')
df['Label_Quality'] = df['Label_Quality'].astype('category')
df['Diagnosis'] = df['Diagnosis'].astype('category')
df['Category'] = df['Category'].astype('category')
df.dtypes

Patient_ID                 int64
Date_MRI          datetime64[ns]
ID_MRI_Machine          category
Adenoma_size            category
Ausfälle prä              object
Label_Quality           category
Entry_date        datetime64[ns]
Operation_date    datetime64[ns]
Ausfälle post             object
Diagnosis               category
Category                category
Patient_age                int64
Prolactin                float64
IGF1                     float64
Cortisol                  object
fT4                       object
Lab_additional            object
dtype: object

In [72]:
# replace and correct wrong namings from labelers
df["Ausfälle prä"]= df["Ausfälle prä"].str.replace("intak","intakt")
df["Ausfälle prä"]= df["Ausfälle prä"].str.replace("goando","gonado")
df["Ausfälle post"]= df["Ausfälle post"].str.replace("goando","gonado")
df["Ausfälle post"]= df["Ausfälle post"].str.replace("adh","ADH")

## One Hot Encode Categorical Values

To use and analyse the categorical data we need to one-hot encode them. This is done by splitting the comma separated strings into single strings and then create a one-hot-encoded column of each individual value. This column is then added to the original dataframe.

In [73]:
# Split the 'Ausfälle prä' column into separate strings
df['Ausfälle prä'] = df['Ausfälle prä'].str.split(', ')
# Create a set to store all unique disfunctions
unique_disfunctions = set()

# Iterate over the 'Ausfälle prä' column to gather unique disfunctions
for value in df['Ausfälle prä']:
    if isinstance(value, list):
        unique_disfunctions.update(value)
    elif isinstance(value, str):
        unique_disfunctions.add(value)

# Iterate over the unique disfunctions and create one-hot encoded columns
for disfunction in unique_disfunctions:
    df["Pre_OP_hormone_"+ disfunction] = df['Ausfälle prä'].apply(lambda x: 1 if (isinstance(x, list) and disfunction in x) or (x == disfunction) else 0)
# drop the original 'Ausfälle prä' column
df = df.drop('Ausfälle prä', axis=1)

In [74]:
# Split the 'Ausfälle post' column into separate strings
df['Ausfälle post'] = df['Ausfälle post'].str.split(', ')

# Create a set to store all unique disfunctions
unique_disfunctions = set()

# Iterate over the 'Ausfälle post' column to gather unique disfunctions
for value in df['Ausfälle post']:
    if isinstance(value, list):
        unique_disfunctions.update(value)
    elif isinstance(value, str):
        unique_disfunctions.add(value)

# Iterate over the unique disfunctions and create one-hot encoded columns
for disfunction in unique_disfunctions:
    df["Post_OP_hormone_"+ disfunction] = df['Ausfälle post'].apply(lambda x: 1 if (isinstance(x, list) and disfunction in x) or (x == disfunction) else 0)

# drop the original 'Ausfälle post' column
df = df.drop('Ausfälle post', axis=1)

In [75]:
df.columns

Index(['Patient_ID', 'Date_MRI', 'ID_MRI_Machine', 'Adenoma_size',
       'Label_Quality', 'Entry_date', 'Operation_date', 'Diagnosis',
       'Category', 'Patient_age', 'Prolactin', 'IGF1', 'Cortisol', 'fT4',
       'Lab_additional', 'Pre_OP_hormone_', 'Pre_OP_hormone_gondao',
       'Pre_OP_hormone_coritco', 'Pre_OP_hormone_intakt',
       'Pre_OP_hormone_morbus cushing', 'Pre_OP_hormone_SIADH',
       'Pre_OP_hormone_cortico', 'Pre_OP_hormone_hypothyreo',
       'Pre_OP_hormone_hyperprolaktin stressbedingt',
       'Pre_OP_hormone_hypogonado', 'Pre_OP_hormone_hyperprolaktin',
       'Pre_OP_hormone_somato', 'Pre_OP_hormone_gonado',
       'Pre_OP_hormone_hyperprolakin stressinduziert', 'Pre_OP_hormone_keine',
       'Pre_OP_hormone_thyreo', 'Pre_OP_hormone_Kompression',
       'Pre_OP_hormone_inaktiv', 'Post_OP_hormone_intakt',
       'Post_OP_hormone_Proliferationsmarker nicht nachgewiesen',
       'Post_OP_hormone_cortico', 'Post_OP_hormone_hyperprolaktin',
       'Post_OP_hormone

In [76]:
df= df[['Patient_ID', 'Date_MRI', 'ID_MRI_Machine','Entry_date', 'Operation_date', 'Adenoma_size',
         'Diagnosis',
       'Category', 'Patient_age', 'Prolactin',"IGF1", 'Cortisol','fT4','Lab_additional',
       'Pre_OP_hormone_cortico', 'Pre_OP_hormone_gonado',
       'Pre_OP_hormone_somato', 'Pre_OP_hormone_thyreo',
       'Pre_OP_hormone_hyperprolaktin', 'Pre_OP_hormone_keine',
       'Pre_OP_hormone_intakt', 'Post_OP_hormone_cortico',
       'Post_OP_hormone_gonado', 'Post_OP_hormone_somato',
       'Post_OP_hormone_ADH', 'Post_OP_hormone_thyreo',
       'Post_OP_hormone_hyperprolaktin', 'Post_OP_hormone_keine',
       'Post_OP_hormone_intakt', 'Label_Quality',]]

In [77]:
df.to_csv(r'../raw_data/label_data.csv',index=False)

In [78]:
print("End Clean and Preprocessing patient data")

End Clean and Preprocessing patient data


# Data Cleaning and Selection Labor data

In [79]:
print("Start Clean and Preprocessing labor data")

Start Clean and Preprocessing labor data


## Read

In [162]:
labor_data_raw = pd.read_excel("../raw_data/labor_data.xlsx")
labor_data = pd.DataFrame({})

In [163]:
# change float with , to float with .
labor_data[labor_data_raw.columns[0]] = [re.sub('"([^",]+),([^"]+)"',r"\1.\2", re.sub('"([^",]+),([^",]+),([^"]+)"',r"\1.\2.\3", string[0])) for string in labor_data_raw.values]

In [164]:
labor_data= labor_data.iloc[:,0].str.split(",",expand=True)
labor_data.columns = labor_data_raw.columns[0].split(",")

In [165]:
labor_data[labor_data=="NULL"] = np.nan

In [166]:
ids = {'Ã¼': 'ü', 'Ã¤': 'ä', "Ã„":"Ä"}

for column in labor_data.columns:
    for old, new in ids.items():
        labor_data[column] = labor_data[column].str.replace(old, new, regex=False)
        
labor_data["Normwert"] = labor_data["Normwert"].str.lower().str.replace(" ", "").replace("",np.nan)
labor_data["Warnung"] = labor_data["Warnung"].replace('   ',np.nan)
labor_data["Datum_Resultat"] = pd.to_datetime(labor_data["Datum_Resultat"],dayfirst=True)



In [199]:
labor_data = labor_data.rename(columns={"FALL_NR":"Case_ID"})
labor_data_for_modell = labor_data.drop(columns = ["PATIENT_NR","Warnung","Einheit","Auftraggeber","Normwert","Datum_Resultat","Analyse"])
#labor_data["Case_ID"] = labor_data["Case_ID"].str.strip("#").astype(int)

In [200]:
#TODO dont delete < 0.2
labor_data_for_modell["Resultat"] = labor_data_for_modell["Resultat"].apply(pd.to_numeric, errors='coerce')
labor_data_for_modell = labor_data_for_modell.dropna(subset = ['Resultat'])

In [201]:
#TODO check for more komplex strategie and add n_labortest
labor_data_for_modell = labor_data_for_modell.groupby(["Case_ID","Analyse-ID","Fallart"])["Resultat"].agg([np.mean,len]).reset_index().rename(columns={"mean":"Resultat","len":"n_labortest"})

In [202]:
labor_data_for_modell

,Case_ID,Analyse-ID,Fallart,Resultat,n_labortest
0,#2017851,FT4,no,13.300000,1
1,#2024754,FT4,no,28.540909,22
2,#2024754,TEST,no,24.088636,22
3,#2024755,20396,no,28.238095,21
4,#2024755,FSH,no,26.431818,22
...,...,...,...,...,...
67325,562762,FT4,no,10.600000,1
67326,70033543,FSH,no,2.710000,1
67327,70033543,LH,no,1.760000,1
67328,70033543,TEST,no,16.700000,1


In [203]:
labor_data_for_modell = labor_data_for_modell.pivot(index=["Case_ID"],values = ['Resultat'], columns = ['Analyse-ID'])
labor_data_for_modell.columns = labor_data_for_modell.columns.droplevel()
labor_data_for_modell = labor_data_for_modell.reset_index()

In [210]:
labor_data_for_modell

Analyse-ID,Case_ID,20396,24382,24383,24384,COR30,COR60,FSH,FT4,LH,PROL,TBILHB,TEST
0,#2017851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.300000,NaN,NaN,NaN,NaN
1,#2024754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.540909,NaN,NaN,NaN,24.088636
2,#2024755,28.238095,NaN,NaN,NaN,NaN,NaN,26.431818,NaN,38.00,22.327273,NaN,NaN
3,#2025084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.150000,NaN,NaN
4,#2025918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.000000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41747,42005575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.600000,NaN,NaN,NaN,NaN
41748,42005643,NaN,NaN,NaN,NaN,NaN,NaN,8.850000,16.700000,NaN,1.500000,NaN,NaN
41749,562762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.600000,NaN,NaN,NaN,NaN
41750,70033543,NaN,NaN,NaN,NaN,NaN,NaN,2.710000,NaN,1.76,NaN,NaN,16.700000


In [184]:
print(f"Sparsity of labordata: {round(labor_data_for_modell.isna().mean().mean(),3)} %")

Sparsity of labordata: 0.866 %


In [205]:
labor_data_for_modell.to_csv(r'../raw_data/labor_data_preprocessed.csv',index=False)

In [92]:
print("End Clean and Preprocessing labor data")

End Clean and Preprocessing labor data


In [142]:
values = []
for value in labor_data["Resultat"]:
    try:
        float(value)
    except:
        values.append(value)

In [143]:
values

['<0.20',
 '<0.20',
 '<0.20',
 '1Â\xa0264',
 '<0.20',
 '<0.30',
 '<0.20',
 '<0.20',
 '<0.45',
 '<0.30',
 '<0.20',
 '<0.20',
 '<0.45',
 '<0.20',
 '<0.20',
 '<0.20',
 '<0.45',
 '<0.20',
 '<0.20',
 '1Â\xa0019',
 '<0.45',
 '<0.20',
 '<0.45',
 '1Â\xa0416',
 '<0.20',
 '<0.45',
 '<0.20',
 '<0.45',
 '<0.20',
 '<0.20',
 '<0.30',
 '<0.20',
 '<0.20',
 '<0.45',
 '<0.20',
 '<0.45',
 '<0.20',
 '1Â\xa0029',
 '<0.20',
 '<0.30',
 '<0.30',
 '<0.45',
 '<0.20',
 '1Â\xa0515',
 '<0.20',
 '<0.45',
 '<0.30',
 '<0.20',
 '<0.20',
 '<0.30',
 '<0.20',
 '<0.20',
 '<0.20',
 '<0.45',
 '<0.20',
 '<0.20',
 '<0.45',
 '<0.20',
 '>64.0',
 '<0.45',
 '<0.20',
 '<0.20',
 '<0.20',
 '<0.45',
 '<0.45',
 '1Â\xa0304',
 '<0.20',
 '1Â\xa0050',
 '<0.20',
 '<0.45',
 '<0.20',
 '<0.20',
 '<0.30',
 '<0.45',
 '<0.20',
 '<0.20',
 '<0.30',
 '<0.20',
 '<0.45',
 '<0.45',
 '<0.45',
 '<0.45',
 '1Â\xa0169',
 '<0.45',
 '<0.20',
 '<0.20',
 '<0.45',
 '1Â\xa0384',
 '<2.00',
 '<0.30',
 '<0.20',
 '>64.0',
 '<0.20',
 '<0.30',
 '<0.45',
 '<0.20',
 '>6

In [94]:
np.unique(np.array(values))

array(['1Â\xa0003', '1Â\xa0004', '1Â\xa0005', '1Â\xa0006', '1Â\xa0007',
       '1Â\xa0009', '1Â\xa0011', '1Â\xa0016', '1Â\xa0017', '1Â\xa0018',
       '1Â\xa0019', '1Â\xa0021', '1Â\xa0027', '1Â\xa0028', '1Â\xa0029',
       '1Â\xa0031', '1Â\xa0034', '1Â\xa0035', '1Â\xa0038', '1Â\xa0041',
       '1Â\xa0043', '1Â\xa0045', '1Â\xa0047', '1Â\xa0049', '1Â\xa0050',
       '1Â\xa0051', '1Â\xa0052', '1Â\xa0053', '1Â\xa0055', '1Â\xa0058',
       '1Â\xa0059', '1Â\xa0061', '1Â\xa0066', '1Â\xa0070', '1Â\xa0071',
       '1Â\xa0075', '1Â\xa0076', '1Â\xa0079', '1Â\xa0082', '1Â\xa0083',
       '1Â\xa0084', '1Â\xa0085', '1Â\xa0087', '1Â\xa0088', '1Â\xa0089',
       '1Â\xa0093', '1Â\xa0094', '1Â\xa0096', '1Â\xa0097', '1Â\xa0098',
       '1Â\xa0100', '1Â\xa0104', '1Â\xa0105', '1Â\xa0113', '1Â\xa0116',
       '1Â\xa0118', '1Â\xa0119', '1Â\xa0121', '1Â\xa0125', '1Â\xa0126',
       '1Â\xa0133', '1Â\xa0136', '1Â\xa0145', '1Â\xa0146', '1Â\xa0147',
       '1Â\xa0149', '1Â\xa0152', '1Â\xa0156', '1Â\xa0159', '1Â\x

In [95]:
labor_data[labor_data["Resultat"] == "1Â\xa0896"]

,Case_ID,Analyse,Analyse-ID,Datum_Resultat,Resultat,Normwert,Fallart
17541,10308286,Cortisol-30' Synacthen-Test,COR30,2021-03-11,1Â 896,>550,no
